In [1]:
import numpy as np
import pandas as pd

In [1]:
"""데이터 출처 :https://www.kaggle.com/rsrishav/youtube-trending-video-dataset?select=KR_youtube_trending_data.csv
데이터 설명 : 유튜브 데일리 인기동영상 (한국)
dataurl : https://raw.githubusercontent.com/Datamanim/datarepo/main/youtube/youtube.csv"""
# Q1. 인기동영상 제작횟수가 많은 채널 상위 10개명을 출력하라 (날짜기준, 중복포함)   ★

df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/youtube/youtube.csv")
df.head(2)

NameError: name 'pd' is not defined

In [48]:
df.groupby(['channelTitle', 'trending_date2']).size().groupby('channelTitle').size().sort_values(ascending=False).head(10).index

Index(['짤툰', '파뿌리', '장삐쭈', '런닝맨 - 스브스 공식 채널', '워크맨-Workman', '육식맨 YOOXICMAN',
       '총몇명', 'SPOTV', '엠뚜루마뚜루 : MBC 공식 종합 채널', 'SMTOWN'],
      dtype='object', name='channelTitle')

In [46]:
# 이건 전체 기간에서 제작 횟수잖아 멍청아;
df.groupby(['channelTitle']).size().sort_values(ascending=False).head(10).index
df.loc[df.channelId.isin(df.channelId.value_counts().head(10).index)].channelTitle.unique() # 왜 돌아감?

Index(['짤툰', '파뿌리', 'SPOTV', '런닝맨 - 스브스 공식 채널', '엠뚜루마뚜루 : MBC 공식 종합 채널', '장삐쭈',
       'BANGTANTV', '채널 십오야', '이과장', '총몇명'],
      dtype='object', name='channelTitle')

In [57]:
# Q2. 논란으로 인기동영상이 된 케이스를 확인하고 싶다. dislikes수가 like 수보다 높은 동영상을 제작한 채널을 모두 출력하라
# ★
df.loc[df.likes < df.dislikes]['channelTitle'].unique()

array(['핫도그TV', 'ASMR 애정TV', '하얀트리HayanTree', '양팡 YangPang',
       '철구형 (CHULTUBE)', '왜냐맨하우스', '(MUTUBE)와꾸대장봉준', '오메킴TV', '육지담',
       'MapleStory_KR', 'ROAD FIGHTING CHAMPIONSHIP', '사나이 김기훈',
       '나혼자산다 STUDIO', 'Gen.G esports'], dtype=object)

In [76]:
# Q3. 채널명을 바꾼 케이스가 있는지 확인하고 싶다.              ★
# channelId의 경우 고유값이므로 이를 통해 채널명을 한번이라도 바꾼 채널의 갯수를 구하여라

df.groupby(['channelId', 'channelTitle']).size().groupby('channelId').size().value_counts()[1:].sum()
df[['channelId', 'channelTitle']].drop_duplicates()['channelId'].value_counts().value_counts()[1:].sum()

71

In [87]:
# Q4. 일요일에 인기있었던 영상들중 가장많은 영상 종류(categoryId)는 무엇인가?

df['mydate'] = pd.to_datetime(df['trending_date2']) # 0월 1화 2수 3목 4금 5토 6일
df.loc[df['mydate'].dt.weekday == 6]['categoryId'].value_counts().index[0]

24

In [90]:
# Q5. 각 요일별 인기 영상들의 categoryId는 각각 몇개 씩인지 하나의 데이터 프레임으로 표현하라

df['dayname'] = df['mydate'].dt.day_name()
df.pivot_table(index='dayname', columns='categoryId', aggfunc='size')

categoryId,1,2,10,15,17,19,20,22,23,24,25,26,27,28,29
dayname,,,,,,,,,,,,,,,
Friday,243,120,833,187,633,90,283,1288,568,2976,444,369,183,171,12
Monday,263,105,837,215,668,92,298,1373,594,3148,453,378,205,160,10
Saturday,255,119,776,198,592,87,296,1289,570,3066,422,364,183,173,10
Sunday,274,99,830,217,636,91,289,1337,556,3096,437,363,199,167,9
Thursday,246,128,890,207,682,92,282,1341,560,2954,470,375,194,166,13
Tuesday,257,119,894,208,708,89,285,1375,569,3084,452,394,194,161,11
Wednesday,234,129,917,207,706,85,291,1333,566,3090,468,385,212,165,12


In [116]:
# Q6. 댓글의 수로 (comment_count) 영상 반응에 대한 판단을 할 수 있다.                   ★
# viewcount대비 댓글수가 가장 높은 영상을 확인하라 (view_count값이 0인 경우는 제외한다)

ans = df.loc[df['view_count'] > 0]
ans['vc'] = ans.comment_count / ans.view_count
ans.sort_values('vc', ascending=False).dropna().head(1).title.values[0]

C:\Users\ALT\AppData\Local\Temp\ipykernel_8828\1344392111.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ans['vc'] = ans.comment_count / ans.view_count


'60분 동안 댓글이 달리지 않으면, 영상이 삭제됩니다. (챌린지)'

In [122]:
# Q7. 댓글의 수로 (comment_count) 영상 반응에 대한 판단을 할 수 있다.
# viewcount대비 댓글수가 가장 낮은 영상을 확인하라 (view_counts, ratio값이 0인경우는 제외한다.)

ans = df.loc[df['view_count'] > 0].loc[df['comment_count'] > 0]
ans['vc'] = ans.comment_count / ans.view_count
ans.sort_values('vc').dropna().head(1).title.values[0]

'Join the BTS #PermissiontoDance Challenge only on YouTube #Shorts'

In [128]:
# Q8. like 대비 dislike의 수가 가장 적은 영상은 무엇인가? (like, dislike 값이 0인경우는 제외한다)

ans = df.loc[df['likes'] > 0].loc[df['dislikes'] > 0]
ans['ld'] = ans.likes / ans.dislikes
ans.sort_values('ld', ascending=False).head(1).title.values[0]

'[줌터뷰] *최초공개* 사부작즈🐰🐶의 비공식 이름은 아이라인즈? 꿀조합 티키타카 가득한 NCT 127 도영&정우의 줌터뷰'

In [137]:
# Q9. 가장많은 트렌드 영상을 제작한 채널의 이름은 무엇인가? (날짜기준, 중복포함)

df.groupby('channelTitle').size().sort_values(ascending=False).index[0]

'짤툰'

In [160]:
# Q10. 20회(20일)이상 인기동영상 리스트에 포함된 동영상의 숫자는?

ans = df[['title', 'channelId']].value_counts().to_frame()
len(ans.loc[ans['count'] >= 20])
(df[['title', 'channelId']].value_counts() >= 20).sum()

40

In [162]:
"""데이터 출처 :https://www.kaggle.com/kukuroo3/youtube-episodic-contents-kr(참고, 데이터 수정)
데이터 설명 : 유튜브 “공범” 컨텐츠 동영상 정보 ( 10분 간격 수집)
dataurl1 (비디오 정보) = https://raw.githubusercontent.com/Datamanim/datarepo/main/youtube/videoInfo.csv
dataurl2 (참가자 채널 정보)= https://raw.githubusercontent.com/Datamanim/datarepo/main/youtube/channelInfo.csv"""

video = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/youtube/videoInfo.csv")
channel = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/youtube/channelInfo.csv")

display(video.head())
display(channel.head())

,videopk,viewcnt,likecnt,dislikecnt,favoritecnt,cmcnt,ct,videoname
0,c5JQp6xafqc,1667010,30474,706,0,6587,2021-10-10 15:20:03,공범 EP1
1,c5JQp6xafqc,1669089,30495,707,0,6589,2021-10-10 15:30:03,공범 EP1
2,c5JQp6xafqc,1674759,30522,711,0,6596,2021-10-10 15:40:02,공범 EP1
3,c5JQp6xafqc,1677026,30555,712,0,6604,2021-10-10 15:50:03,공범 EP1
4,c5JQp6xafqc,1681824,30585,713,0,6600,2021-10-10 16:00:03,공범 EP1


,channelid,subcnt,viewcnt,videocnt,ct,channelname
0,UCkQCwnkQfgSuPTTnw_Y7v7w,1310000,410238653,736,2021-09-30 03:01:03,꽈뚜룹
1,UCkQCwnkQfgSuPTTnw_Y7v7w,1310000,412531322,736,2021-09-30 09:01:03,꽈뚜룹
2,UCkQCwnkQfgSuPTTnw_Y7v7w,1310000,412531322,735,2021-09-30 15:01:03,꽈뚜룹
3,UCkQCwnkQfgSuPTTnw_Y7v7w,1310000,412531322,737,2021-09-30 21:01:03,꽈뚜룹
4,UCkQCwnkQfgSuPTTnw_Y7v7w,1320000,412531322,737,2021-10-01 03:01:04,꽈뚜룹


In [166]:
# Q11. 각 데이터의 ‘ct’컬럼을 시간으로 인식할수 있게 datatype을 변경하고
# video 데이터의 videoname의 각 value 마다 몇개의 데이터씩 가지고 있는지 확인하라

video['ct'] = pd.to_datetime(video['ct'])
channel['ct'] = pd.to_datetime(channel['ct'])

display(video.info())
video.groupby('videoname').size()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15201 entries, 0 to 15200
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   videopk      15201 non-null  object        
 1   viewcnt      15201 non-null  int64         
 2   likecnt      15201 non-null  int64         
 3   dislikecnt   15201 non-null  int64         
 4   favoritecnt  15201 non-null  int64         
 5   cmcnt        15201 non-null  int64         
 6   ct           15201 non-null  datetime64[ns]
 7   videoname    15201 non-null  object        
dtypes: datetime64[ns](1), int64(5), object(2)
memory usage: 950.2+ KB


None

videoname
공범 EP1    3492
공범 EP2    3204
공범 EP3    2568
공범 EP4    2280
공범 EP5    1562
공범 EP6    1274
공범 EP7     555
공범 EP8     266
dtype: int64

In [169]:
# Q12. 수집된 각 video의 가장 최신화 된 날짜의 viewcount값을 출력하라

video[['videoname', 'viewcnt', 'ct']].groupby('videoname').max()
#video.sort_values(['videoname','ct']).drop_duplicates('videoname',keep='last')[['viewcnt','videoname','ct']].reset_index(drop=True)

,viewcnt,ct
videoname,,
공범 EP1,3180532,2021-11-01 15:30:03
공범 EP2,2199328,2021-11-01 15:30:03
공범 EP3,1671294,2021-11-01 15:30:03
공범 EP4,1818493,2021-11-01 15:30:03
공범 EP5,1503435,2021-11-01 15:30:04
공범 EP6,1750222,2021-11-01 15:30:04
공범 EP7,1630200,2021-11-01 15:30:05
공범 EP8,1289088,2021-11-01 15:30:05


In [185]:
# Q13. Channel 데이터중 2021-10-03일 이후 각 채널의 처음 기록 됐던 구독자 수(subcnt)를 출력하라     ★

ans = channel[['channelname', 'ct', 'subcnt']].loc[channel['ct'] >= pd.to_datetime('2021-10-04')]
ans.sort_values(['channelname', 'ct']).drop_duplicates('channelname').drop('ct', axis=1).reset_index(drop=True)

,channelname,subcnt
0,Balming Tiger,54400
1,곽토리 kwak tori,471000
2,김농밀의 농밀한 삶,7530
3,꽈뚜룹,1330000
4,논리왕 전기,921000
5,릴펄 Lilpearl,10100
6,야전삽짱재,257000
7,와글와글 WagleWagle,55000
8,조나단,215000
9,츄정ChuJeong,322000


In [225]:
# Q14. 각채널의 2021-10-03 03:00:00 ~ 2021-11-01 15:00:00 까지 구독자수 (subcnt) 의 증가량을 구하여라   ★

start = channel[['channelname', 'ct', 'subcnt']].loc[channel['ct'] >= pd.to_datetime('2021-10-03 03')].drop_duplicates('channelname')
display(start)
end = channel[['channelname', 'ct', 'subcnt']].loc[channel['ct'] >= pd.to_datetime('2021-11-01 15')].drop_duplicates('channelname')
display(end)
start = start[['channelname', 'subcnt']].reset_index(drop=True)
end = end[['channelname', 'subcnt']].reset_index(drop=True)
ans = pd.merge(start, end, on='channelname')
ans['dif'] = ans['subcnt_y'] - ans['subcnt_x']
ans.drop(['subcnt_y', 'subcnt_x'], axis=1)

,channelname,ct,subcnt
12,꽈뚜룹,2021-10-03 03:01:04,1330000
139,야전삽짱재,2021-10-03 03:01:02,257000
266,츄정ChuJeong,2021-10-03 03:01:02,322000
393,논리왕 전기,2021-10-03 03:01:02,922000
520,와글와글 WagleWagle,2021-10-03 03:01:03,55000
647,조나단,2021-10-03 03:01:03,215000
774,형사!탐정되다,2021-10-03 03:01:03,14900
901,김농밀의 농밀한 삶,2021-10-03 03:01:03,7520
1028,릴펄 Lilpearl,2021-10-03 03:01:03,10100
1155,곽토리 kwak tori,2021-10-03 03:01:04,471000


,channelname,ct,subcnt
130,꽈뚜룹,2021-11-01 15:01:08,1400000
257,야전삽짱재,2021-11-01 15:01:03,268000
384,츄정ChuJeong,2021-11-01 15:01:03,323000
511,논리왕 전기,2021-11-01 15:01:04,911000
638,와글와글 WagleWagle,2021-11-01 15:01:04,55000
765,조나단,2021-11-01 15:01:04,227000
892,형사!탐정되다,2021-11-01 15:01:04,25200
1019,김농밀의 농밀한 삶,2021-11-01 15:01:05,9060
1146,릴펄 Lilpearl,2021-11-01 15:01:05,21100
1273,곽토리 kwak tori,2021-11-01 15:01:05,469000


,channelname,dif
0,꽈뚜룹,70000
1,야전삽짱재,11000
2,츄정ChuJeong,1000
3,논리왕 전기,-11000
4,와글와글 WagleWagle,0
5,조나단,12000
6,형사!탐정되다,10300
7,김농밀의 농밀한 삶,1540
8,릴펄 Lilpearl,11000
9,곽토리 kwak tori,-2000


In [284]:
# Q15. 각 비디오는 10분 간격으로 구독자수, 좋아요, 싫어요수, 댓글수가 수집된것으로 알려졌다.
# 공범 EP1의 비디오정보 데이터중 수집간격이 5분 이하, 20분이상인 데이터 구간( 해당 시점 전,후) 의 시각을 모두 출력하라

cut = video.loc[video['videoname'] == ' 공범 EP1'].sort_values('ct').reset_index(drop=True)
dif = cut['ct'].diff().to_frame()
import datetime
res = dif[(dif['ct'] <= datetime.timedelta(minutes=5)) | (dif['ct'] >= datetime.timedelta(minutes=20))]
ans = sorted(set(res.index.to_list() + [i-1 for i in res.index.to_list()]))
cut.iloc[ans]

,videopk,viewcnt,likecnt,dislikecnt,favoritecnt,cmcnt,ct,videoname
720,c5JQp6xafqc,2153676,34104,830,0,7120,2021-10-12 19:20:03,공범 EP1
721,c5JQp6xafqc,2228250,34559,849,0,7191,2021-10-13 09:41:37,공범 EP1
722,c5JQp6xafqc,2228250,34559,849,0,7191,2021-10-13 09:41:37,공범 EP1
1635,c5JQp6xafqc,2706691,37893,987,0,7410,2021-10-19 17:50:02,공범 EP1
1636,c5JQp6xafqc,2707933,37901,988,0,7410,2021-10-19 18:20:03,공범 EP1


In [304]:
# Q16. 각 에피소드의 시작날짜(년-월-일)를 에피소드 이름과 묶어 데이터 프레임으로 만들고 출력하라

ans = video[['videoname', 'ct']].sort_values('ct')
ans['ct'] = ans['ct'].dt.date
ans.drop_duplicates('videoname').set_index('videoname')

,ct
videoname,
공범 EP1,2021-10-07
공범 EP2,2021-10-09
공범 EP3,2021-10-14
공범 EP4,2021-10-16
공범 EP5,2021-10-21
공범 EP6,2021-10-23
공범 EP7,2021-10-28
공범 EP8,2021-10-30


In [311]:
# Q17. “공범” 컨텐츠의 경우 19:00시에 공개 되는것으로 알려져있다.
# 공개된 날의 21시의 viewcnt, ct, videoname 으로 구성된 데이터 프레임을 viewcnt를 내림차순으로 정렬하여 출력하라

ans = video.loc[video['ct'].dt.hour == 21][['videoname', 'ct', 'viewcnt']].sort_values('ct')
ans.drop_duplicates('videoname').sort_values('viewcnt', ascending=False).reset_index(drop=True)

,videoname,ct,viewcnt
0,공범 EP8,2021-10-30 21:00:08,264029
1,공범 EP7,2021-10-28 21:00:04,252032
2,공범 EP4,2021-10-16 21:00:04,217674
3,공범 EP6,2021-10-23 21:00:06,213899
4,공범 EP5,2021-10-21 21:00:04,201179
5,공범 EP2,2021-10-09 21:00:03,148144
6,공범 EP3,2021-10-14 21:00:04,147183
7,공범 EP1,2021-10-07 21:00:03,117340


In [318]:
# Q18. video 정보의 가장 최근 데이터들에서 각 에피소드의 싫어요/좋아요 비율을 ratio 컬럼으로 만들고
# videoname, ratio로 구성된 데이터 프레임을 ratio를 오름차순으로 정렬하라

ans = video.sort_values('ct', ascending=False).drop_duplicates('videoname')
ans['ratio'] = ans.dislikecnt / ans.likecnt
ans.sort_values('ratio')

,videopk,viewcnt,likecnt,dislikecnt,favoritecnt,cmcnt,ct,videoname,ratio
13660,QUjQbgj5IXM,1630200,29335,400,0,18247,2021-11-01 15:30:05,공범 EP7,0.013636
14934,AxGKHmXyuAE,1750222,24494,389,0,13025,2021-11-01 15:30:04,공범 EP6,0.015881
11543,bK0_3Vbfvs8,1818493,22703,408,0,16949,2021-11-01 15:30:03,공범 EP4,0.017971
13105,hv7FBjskAu0,1503435,19926,421,0,12434,2021-11-01 15:30:04,공범 EP5,0.021128
9263,GygoAk1hDU0,1671294,22719,508,0,12499,2021-11-01 15:30:03,공범 EP3,0.022360
3491,c5JQp6xafqc,3180532,41043,1113,0,7674,2021-11-01 15:30:03,공범 EP1,0.027118
6695,23QQ1ru9YQg,2199328,25566,763,0,9171,2021-11-01 15:30:03,공범 EP2,0.029844
15200,yZt-h-KcmUE,1289088,25616,3621,0,31663,2021-11-01 15:30:05,공범 EP8,0.141357


In [330]:
# Q19. 2021-11-01 00:00:00 ~ 15:00:00까지 각 에피소드별 viewcnt의 증가량을 데이터 프레임으로 만드시오

start = video.loc[video['ct'] >= pd.to_datetime('2021-11-01 00')].drop_duplicates('videoname').sort_values('videoname')[['videoname', 'viewcnt', 'ct']]
end = video.loc[video['ct'] >= pd.to_datetime('2021-11-01 15')].drop_duplicates('videoname').sort_values('videoname')[['videoname', 'viewcnt', 'ct']]
ans = pd.merge(start, end, on='videoname')
ans['increament'] = ans.viewcnt_y - ans.viewcnt_x
ans.drop(['viewcnt_x', 'viewcnt_y', 'ct_x', 'ct_y'], axis=1)

,videoname,increament
0,공범 EP1,13571
1,공범 EP2,10493
2,공범 EP3,10042
3,공범 EP4,9986
4,공범 EP5,10824
5,공범 EP6,14259
6,공범 EP7,27353
7,공범 EP8,90049


In [340]:
# Q20. video 데이터 중에서 중복되는 데이터가 존재한다. 중복되는 각 데이터의 시간대와 videoname 을 구하여라

ans = video[['videoname', 'ct']].value_counts().to_frame()
ans.loc[ans['count'] > 1].reset_index().drop('count', axis=1)

,videoname,ct
0,공범 EP2,2021-10-13 09:41:37
1,공범 EP1,2021-10-13 09:41:37


In [342]:
"""데이터 출처 :https://www.kaggle.com/darinhawley/fifa-world-cup-goalscorers-19302018(참고, 데이터 수정)
데이터 설명 : 1930 ~2018년도 월드컵 출전선수 골기록
data url = https://raw.githubusercontent.com/Datamanim/datarepo/main/worldcup/worldcupgoals.csv"""

df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/worldcup/worldcupgoals.csv")
df.head(3)

,Player,Goals,Years,Country
0,Miroslav Klose,16,2002-2006-2010-2014,Germany
1,Ronaldo,15,1998-2002-2006,Brazil
2,Gerd Muller,14,1970-1974,Germany


In [347]:
# Q21. 주어진 전체 기간의 각 나라별 골득점수 상위 5개 국가와 그 득점수를 데이터프레임형태로 출력하라

df[['Country', 'Goals']].groupby('Country').sum().sort_values('Goals', ascending=False).head()

,Goals
Country,
Brazil,228
Germany,226
Argentina,135
Italy,123
France,115


In [367]:
# Q22. 주어진 전체기간동안 골득점을 한 선수가 가장 많은 나라 상위 5개 국가와 그 선수 숫자를 데이터 프레임 형식으로 출력하라
# ★
df.groupby(['Country', ]).size().sort_values(ascending=False).head()
#df.groupby('Country').size().sort_values(ascending=False).head(5)
# ans = df.loc[df['Country'] == 'Brazil']
# ans.loc[df['Player'] == 'Junior']      # 아 동명이인이 있네

Country
Brazil       81
Germany      78
Italy        60
Argentina    59
France       58
dtype: int64

In [382]:
# Q23. Years 컬럼은 년도 -년도 형식으로 구성되어있고, 각 년도는 4자리 숫자이다.
# 년도 표기가 4자리 숫자로 안된 케이스가 존재한다. 해당 건은 몇건인지 출력하라

ans = 0
indexes = []
for idx, string in enumerate(df['Years'].to_list()):
    ans += (temp:= sum([1 if i != 4 else 0 for i in list(map(len, string.split('-')))]))
    if temp > 0:
        indexes.append(idx)
print(ans)

45


In [388]:
# Q24. Q23에서 발생한 예외 케이스를 제외한 데이터프레임을 df2라고 정의하고 데이터의 행의 숫자를 출력하라
# (아래 문제부터는 df2로 풀이하겠습니다)

df2 = df.drop(indexes).reset_index(drop=True)
df2.shape[0]

1250

In [405]:
# Q25. 월드컵 출전횟수를 나타내는 ‘LenCup’ 컬럼을 추가하고 4회 출전한 선수의 숫자를 구하여라

def lencup(x):
    return len(x.split('-'))
df2['LenCup'] = df2['Years'].apply(lencup)
len(df2.loc[df2['LenCup'] == 4])
df2['LenCup'].value_counts()[4]     # 이게 좀 더 깔끔

16

In [409]:
# Q26. Yugoslavia 국가의 월드컵 출전횟수가 2회인 선수들의 숫자를 구하여라

len(df2.loc[(df2['Country'] == 'Yugoslavia') & (df2['LenCup'] == 2)])

7

In [413]:
# Q27. 2002년도에 출전한 전체 선수는 몇명인가?

sum(df2['Years'].str.contains('2002'))

156

In [432]:
# Q28. 이름에 ‘carlos’ 단어가 들어가는 선수의 숫자는 몇 명인가? (대, 소문자 구분 x)

ans = 0
for s in df2['Player'].str.lower():
    if 'carlos' in s:
        ans += 1
print(ans)
sum(df2['Player'].str.lower().str.contains('carlos'))

13


13

In [439]:
# Q29. 월드컵 출전 횟수가 1회뿐인 선수들 중에서 가장 많은 득점을 올렸던 선수는 누구인가?

df2.loc[df2['LenCup'] == 1].sort_values('Goals').tail(1)['Player'].values[0]

'Just Fontaine'

In [443]:
# Q30. 월드컵 출전횟수가 1회 뿐인 선수들이 가장 많은 국가는 어디인가?

df2.loc[df2['LenCup'] == 1]['Country'].value_counts().index[0]

'Brazil'

In [4]:
"""데이터 출처 :https://www.data.go.kr/data/15051872/fileData.do(참고, 데이터 수정)
데이터 설명 : 서울특별시_공공자전거 시간대별 이용정보
data url = https://raw.githubusercontent.com/Datamanim/datarepo/main/bicycle/seoul_bi.csv"""

df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/bicycle/seoul_bi.csv")
df.head(3)

,대여일자,대여시간,대여소번호,대여구분코드,성별,연령대코드,이용건수,운동량,탄소량,이동거리,사용시간
0,2021-06-01,0,3541,정기권,F,~10대,1,0.00,0.00,0.00,8
1,2021-06-01,0,765,정기권,F,~10대,1,27.21,0.35,1526.81,19
2,2021-06-01,0,2637,정기권,F,~10대,1,41.40,0.37,1608.56,18


In [449]:
# Q31. 대여일자별 데이터의 수를 데이터프레임으로 출력하고, 가장 많은 데이터가 있는 날짜를 출력하라

ans = df['대여일자'].value_counts().to_frame()
display(ans.sort_index())
ans.head(1).index[0]

,count
대여일자,
2021-06-01,48081
2021-06-02,56100
2021-06-03,18985
2021-06-04,56788
2021-06-05,52730
2021-06-06,47787
2021-06-07,52623
2021-06-08,55623
2021-06-09,54507


'2021-06-04'

In [5]:
# Q32. 각 일자의 요일을 표기하고 (‘Monday’ ~’Sunday’) ‘day_name’컬럼을 추가하고 이를 이용하여
# 각 요일별 이용 횟수의 총합을 데이터 프레임으로 출력하라

df['대여일자'] = pd.to_datetime(df['대여일자'])
df['day_name'] = df['대여일자'].dt.day_name()
df.groupby('day_name').size()

day_name
Friday       108877
Monday       107568
Saturday     101299
Sunday        90918
Thursday      55977
Tuesday      103704
Wednesday    110607
dtype: int64

In [466]:
# Q33. 각 요일별 가장 많이 이용한 대여소의 이용횟수와 대여소 번호를 데이터 프레임으로 출력하라

ans = df[['day_name', '대여소번호']].value_counts().to_frame().reset_index()
ans.drop_duplicates('day_name').set_index('day_name')

,대여소번호,count
day_name,,
Saturday,502,378
Sunday,502,372
Wednesday,502,282
Friday,502,277
Tuesday,502,267
Monday,502,242
Thursday,2715,137


In [479]:
# Q34. 나이대별 대여구분 코드의 (일일권/전체횟수) 비율을 구한 후 가장 높은 비율을 가지는 나이대를 확인하라.
# 일일권의 경우 일일권 과 일일권(비회원)을 모두 포함하라

ans = df['연령대코드'].value_counts().to_frame('cnt')
ans['cnt'] = ans['cnt'] / len(df)
display(ans)
ans.head(1).cnt.index[0]

,cnt
연령대코드,
20대,0.364623
30대,0.275016
40대,0.169083
50대,0.103731
~10대,0.054385
60대,0.028409
70대~,0.004753


'20대'

In [477]:
# Q35. 연령대별 평균 이동거리를 구하여라
df['연령대코드'].value_counts()

연령대코드
20대     247561
30대     186722
40대     114799
50대      70428
~10대     36925
60대      19288
70대~      3227
Name: count, dtype: int64

In [21]:
# Q36. 연령대 코드가 20대인 데이터를 추출하고,이동거리값이 추출한 데이터의 이동거리값의 평균 이상인 데이터를 추출한다.
# 최종 추출된 데이터를 대여일자, 대여소 번호 순서로 내림차순 정렬 후 1행부터 200행까지의 탄소량의 평균을 소숫점 3째 자리까지 구하여라

mean_value = df.loc[df['연령대코드']=='20대']['이동거리'].mean()
ans = df.loc[(df['연령대코드']=='20대') & (df['이동거리'] >= mean_value)]
ans.sort_values(['대여일자', '대여소번호'], ascending=False).head(200)['탄소량'].astype(float).mean().__round__(3)

1.613

In [27]:
# Q37. 6월 7일 ~10대의 “이용건수”의 중앙값은?

df.loc[(df['대여일자'] == pd.to_datetime('2021-06-07')) & (df['연령대코드'] == '~10대')]['이용건수'].median()

1.0

In [62]:
# Q38. 평일 (월~금) 출근 시간대(오전 6,7,8시)의 대여소별 이용 횟수를 구해서 데이터 프레임 형태로 표현한 후
# 각 대여시간별 이용 횟수의 상위 3개 대여소와 이용횟수를 출력하라       ★

temp = (temp:=df[['대여일자', '대여시간', '이용건수', '대여소번호', 'day_name']]).loc[~(temp['day_name'].isin(['Saturday', 'Sunday'])) & temp['대여시간'].isin([6, 7, 8]) ]
ans = temp[['대여시간', '대여소번호', '이용건수']].groupby(['대여시간', '대여소번호']).sum() #.size().to_frame('이용건수')
ans.sort_values(['대여시간', '이용건수'], ascending=False).groupby('대여시간').head(3)   # 아니 왜자꾸 행수랑 이용건수랑 혼용함?

이용건수
대여시간 대여소번호      
8    2701    360
     2715    155
     646     151
7    259     184
     2744    125
     2733    104
6    2744     55
     1028     47
     1125     44

In [69]:
# Q39. 이동거리가 평균 이상의 이동거리 값을 가지는 데이터를 추출하여 추출데이터의 이동거리의 표본표준편차 값을 구하여라

mean_value = df['이동거리'].mean()
df['이동거리'].loc[df['이동거리'] >= mean_value].std()

5092.139707505356

In [78]:
# Q40. 남성(‘M’ or ‘m’)과 여성(‘F’ or ‘f’)의 이동거리값의 평균값을 구하여라

df['성별'] = df['성별'].str.upper()
df[['성별', '이동거리']].groupby('성별').mean()

,이동거리
성별,
F,3468.575025
M,3209.110871


In [2]:
"""데이터 출처 :https://www.kaggle.com/unsdsn/world-happiness(참고, 데이터 수정)
데이터 설명 : 전세계 행복도 지표 조사
data url = https://raw.githubusercontent.com/Datamanim/datarepo/main/happy2/happiness.csv"""

df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/happy2/happiness.csv")
df.head(3)

,행복랭킹,나라명,점수,상대GDP,사회적지원,행복기대치,선택의 자유도,관대함,부패에 대한인식,년도
0,1,Finland,7.769,1.340,1.587,0.986,0.596,0.153,0.393,2019
1,2,Denmark,7.600,1.383,1.573,0.996,0.592,0.252,0.410,2019
2,3,Norway,7.554,1.488,1.582,1.028,0.603,0.271,0.341,2019


In [3]:
# Q41. 데이터는 2018년도와 2019년도의 전세계 행복 지수를 표현한다.
# 각년도의 행복랭킹 10위를 차지한 나라의 행복점수의 평균을 구하여라.

display(df[['년도', '점수']].groupby('년도').mean())
df.loc[df['행복랭킹'] == 10]['점수'].mean()

,점수
년도,
2018,5.375917
2019,5.407096


7.259

In [4]:
# Q42. 각년도의 행복랭킹 50위이내의 나라들의 각각의 행복점수 평균을 데이터프레임으로 표시하라

df.loc[df['행복랭킹'] <= 50][['년도', '점수']].groupby('년도').mean()

,점수
년도,
2018,6.64678
2019,6.67002


In [5]:
# Q43. 2018년도 데이터들만 추출하여 행복점수와 부패에 대한 인식에 대한 상관계수를 구하여라      ★

df.loc[df['년도'] == 2018][['점수', '부패에 대한인식']].corr().iloc[0,1]

0.40529152271510027

In [9]:
# Q44. 2018년도와 2019년도의 행복랭킹이 변화하지 않은 나라명의 수를 구하여라

temp = df[['행복랭킹', '나라명']].value_counts().to_frame('count')
len(temp.loc[temp['count'] > 1])

15

In [10]:
# Q45. 2019년도 데이터들만 추출하여 각변수간 상관계수를 구하고 내림차순으로 정렬한 후
# 상위 5개를 데이터 프레임으로 출력하라. 컬럼명은 v1,v2,corr으로 표시하라

temp = df.loc[df['년도'] == 2019].drop('나라명', axis=1).corr().stack().drop_duplicates().to_frame('corr').reset_index()
temp.columns = ['v1', 'v2', 'corr']
temp = temp.loc[temp['corr'] < 1]
temp.sort_values('corr', ascending=False).head()

,v1,v2,corr
15,상대GDP,행복기대치,0.835462
8,점수,상대GDP,0.793883
10,점수,행복기대치,0.779883
9,점수,사회적지원,0.777058
14,상대GDP,사회적지원,0.754906


In [11]:
# Q46. 각 년도별 하위 행복점수의 하위 5개 국가의 평균 행복점수를 구하여라

df[['년도', '점수']].sort_values('점수').groupby('년도').head(5).groupby('년도').mean()

,점수
년도,
2018,3.1800
2019,3.1408


In [ ]:
# Q47. 2019년 데이터를 추출하고 해당데이터의 상대 GDP 평균 이상의 나라들과
# 평균 이하의 나라들의 행복점수 평균을 각각 구하고 그 차이값을 출력하라




In [ ]:
# Q48. 각년도의 부패에 대한인식을 내림차순 정렬했을때 상위 20개 국가의 부패에 대한인식의 평균을 구하여라

In [ ]:
# Q49. 2018년도 행복랭킹 50위 이내에 포함됐다가 2019년 50위 밖으로 밀려난 국가의 숫자를 구하여라

In [ ]:
# Q50. 2018년,2019년 모두 기록이 있는 나라들 중 년도별 행복점수가 가장 증가한 나라와 그 증가 수치는?

In [ ]:
"""데이터 출처 : https://archive.ics.uci.edu/ml/datasets/Power+consumption+of+Tetouan+city (참고, 데이터 수정)
데이터 설명 : 기온, 습도,바람풍속에 따른 도시의 3개 지역구의 에너지 소비량
data url = https://raw.githubusercontent.com/Datamanim/datarepo/main/consum/Tetuan City power consumption.csv"""
# Q51. DateTime컬럼을 통해 각 월별로 몇개의 데이터가 있는지 데이터 프레임으로 구하여라

In [ ]:
# Q52. 3월달의 각 시간대별 온도의 평균들 중 가장 낮은 시간대의 온도를 출력하라

In [ ]:
# Q53. 3월달의 각 시간대별 온도의 평균들 중 가장 높은 시간대의 온도를 출력하라

In [ ]:
# Q54. Zone 1 Power Consumption 컬럼의 value값의 크기가
# Zone 2 Power Consumption 컬럼의 value값의 크기보다 큰 데이터들의 Humidity의 평균을 구하여라

In [ ]:
# Q55. 각 zone의 에너지 소비량의 상관관계를 구해서 데이터 프레임으로 표기하라

In [ ]:
# Q56. Temperature의 값이 10미만의 경우 A, 10이상 20미만의 경우 B,20이상 30미만의 경우 C,
# 그 외의 경우 D라고 할때 각 단계의 데이터 숫자를 구하여라

In [ ]:
# Q57. 6월 데이터중 12시의 Temperature의 표준편차를 구하여라

In [ ]:
# Q58. 6월 데이터중 12시의 Temperature의 분산을 구하여라

In [ ]:
# Q59. Temperature의 평균이상의 Temperature의 값을 가지는 데이터를 Temperature를 기준으로 정렬 했을때
# 4번째 행의 Humidity 값은?

In [ ]:
# Q60. Temperature의 중간값 이상의 Temperature의 값을 가지는 데이터를Temperature를 기준으로 정렬 했을때
# 4번째 행의 Humidity 값은?

In [ ]:
"""데이터 출처 : https://www.kaggle.com/abcsds/pokemon (참고, 데이터 수정)
데이터 설명 : 포켓몬 정보
data url = https://raw.githubusercontent.com/Datamanim/datarepo/main/pok/Pokemon.csv"""
# Q61. Legendary 컬럼은 전설포켓몬 유무를 나타낸다.전설포켓몬과 그렇지 않은 포켓몬들의 HP평균의 차이를 구하여라

In [ ]:
# Q62. Type 1은 주속성 Type 2 는 부속성을 나타낸다. 가장 많은 부속성 종류는 무엇인가?

In [ ]:
# Q63. 가장 많은 Type 1 의 종의 평균 Attack 을 평균 Defense로 나눈값은?

In [ ]:
# Q64. 포켓몬 세대(Generation) 중 가장많은 Legendary를 보유한 세대는 몇세대인가?

In [ ]:
# Q65. ‘HP’, ‘Attack’, ‘Defense’, ‘Sp. Atk’, ‘Sp. Def’, ‘Speed’ 간의 상관 계수중
# 가장 절댓값이 큰 두 변수와 그 값을 구하여라

In [ ]:
# Q66. 각 Generation의 Attack으로 오름차순 정렬시 상위 3개 데이터들(18개)의 Attack의 전체 평균을 구하여라

In [ ]:
# Q67. 각 Generation의 Attack으로 내림차순 정렬시 상위 5개 데이터들(30개)의 Attack의 전체 평균을 구하여라

In [ ]:
# Q68. 가장 흔하게 발견되는 (Type1 , Type2) 의 쌍은 무엇인가?

In [ ]:
# Q69. 한번씩만 존재하는 (Type1 , Type2)의 쌍의 갯수는 몇개인가?

In [ ]:
# Q70. 한번씩만 존재하는 (Type1 , Type2)의 쌍을 각 세대(Generation)은 각각 몇개씩 가지고 있는가?

In [ ]:
"""데이터 출처 : 국민체육진흥공단 (문화 빅데이터플랫폼) (참고, 데이터 수정)
데이터 설명 : 대한민국 국민 체력장 평가
data url = https://raw.githubusercontent.com/Datamanim/datarepo/main/body/body.csv"""
# Q71. 전체데이터의 수축기혈압(최고) - 이완기혈압(최저)의 평균을 구하여라

In [ ]:
# Q72. 50~59세의 신장평균을 구하여라

In [ ]:
# Q73. 연령대 (20~29 : 20대 …) 별 인원수를 구하여라

In [ ]:
# Q74. 연령대 (20~29 : 20대 …) 별 등급의 숫자를 데이터 프레임으로 표현하라

In [ ]:
# Q75. 남성 중 A등급과 D등급의 체지방률 평균의 차이(큰 값에서 작은 값의 차)를 구하여라

In [ ]:
# Q76. 여성 중 A등급과 D등급의 체중의 평균의 차이(큰 값에서 작은 값의 차)를 구하여라

In [ ]:
# Q77. bmi는 자신의 몸무게(kg)를 키의 제곱(m)으로 나눈값이다.
# 데이터의 bmi 를 구한 새로운 컬럼을 만들고 남성의 bmi 평균을 구하여라

In [ ]:
# Q78. bmi보다 체지방율이 높은 사람들의 체중평균을 구하여라

In [ ]:
# Q79. 남성과 여성의 악력 평균의 차이를 구하여라

In [ ]:
# Q80. 남성과 여성의 교차윗몸일으키기 횟수의 평균의 차이를 구하여라

In [ ]:
"""데이터 출처 : https://data.kma.go.kr/cmmn/static/staticPage.do?page=intro
데이터 설명 : 2020년도 이화동(서울) , 수영동(부산)의 시간단위의 기온과 강수량
데이터 url : https://raw.githubusercontent.com/Datamanim/datarepo/main/weather/weather2.csv"""
# Q81. 여름철(6월,7월,8월) 이화동이 수영동보다 높은 기온을 가진 시간대는 몇개인가?

In [ ]:
# Q82. 이화동과 수영동의 최대강수량의 시간대를 각각 구하여라

In [ ]:
"""데이터 출처 : https://www.kaggle.com/shubh0799/churn-modelling 에서 변형
데이터 설명 : 고객의 신상정보 데이터를 통한 회사 서비스 이탈 예측 (종속변수 : Exited)
Data url : https://raw.githubusercontent.com/Datamanim/datarepo/main/churn/train.csv"""
# Q83. 남성 이탈(Exited)이 가장 많은 국가(Geography)는 어디이고 이탈 인원은 몇명인가?

In [ ]:
# Q84. 카드를 소유(HasCrCard ==1)하고 있으면서 활성멤버(IsActiveMember ==1) 인 고객들의
# 평균 나이를 소숫점이하 4자리까지 구하여라?

In [ ]:
# Q85. Balance 값이 중간값 이상을 가지는 고객들의 CreditScore의 표준편차를 소숫점이하 3자리까지 구하여라

In [ ]:
"""데이터 출처 :https://www.data.go.kr/data/15007122/fileData.do (참고, 데이터 수정)
데이터 설명 : 2018년도 성인의 건강검 진데이터 (흡연상태 1- 흡연, 0-비흡연 )
데이터 url : https://raw.githubusercontent.com/Datamanim/datarepo/main/smoke/train.csv"""
# Q86. 수축기혈압과 이완기 혈압기 수치의 차이를 새로운 컬럼(‘혈압차’) 으로 생성하고,
#  연령대 코드별 각 그룹 중 ‘혈압차’ 의 분산이 5번째로 큰 연령대 코드를 구하여라

In [ ]:
# Q87. 비만도를 나타내는 지표인 WHtR는 허리둘레 / 키로 표현한다. 일반적으로 0.58이상이면 비만으로 분류한다.
#  데이터중 WHtR 지표상 비만인 인원의 남/여 비율을 구하여라

In [ ]:
"""데이터 출처 :https://www.kaggle.com/anmolkumar/health-insurance-cross-sell-prediction(참고, 데이터 수정)
데이터 설명 : 자동차 보험 가입 예측
Data url : https://raw.githubusercontent.com/Datamanim/datarepo/main/insurance/train.csv"""
# Q88. Vehicle_Age 값이 2년 이상인 사람들만 필터링 하고 그중에서
#  Annual_Premium 값이 전체 데이터의 중간값 이상인 사람들을 찾고, 그들의 Vintage값의 평균을 구하여라

In [ ]:
# Q89. vehicle_age에 따른 각 성별(gender)그룹의 Annual_Premium값의 평균을 구하여 아래 테이블과 동일하게 구현하라

In [ ]:
"""데이터 출처 :https://www.kaggle.com/iabhishekofficial/mobile-price-classification?select=train.csv(참고, 데이터 수정)
데이터 설명 : 핸드폰 가격예측 (price_range컬럼 0(저렴) ~3(매우비쌈) 범위 )
Data url https://raw.githubusercontent.com/Datamanim/datarepo/main/mobile/train.csv"""
# Q90. price_range 의 각 value를 그룹핑하여 각 그룹의 n_cores 의 빈도가 가장높은 value와 그 빈도수를 구하여라

In [ ]:
# Q91. price_range 값이 3인 그룹에서 상관관계가 2번째로 높은 두 컬럼과 그 상관계수를 구하여라

In [ ]:
"""데이터 출처 :https://www.kaggle.com/teejmahal20/airline-passenger-satisfaction?select=train.csv (참고, 데이터 수정)
데이터 설명 : 비행탑승 경험 만족도 (satisfaction 컬럼 : ‘neutral or dissatisfied’ or satisfied ) (83123, 24) shape
Data url :https://raw.githubusercontent.com/Datamanim/datarepo/main/airline/train.csv"""
# Q92. Arrival Delay in Minutes 컬럼이 결측치인 데이터들 중 ‘neutral or dissatisfied’ 보다
# ‘satisfied’의 수가 더 높은 Class는 어디 인가?

In [ ]:
"""데이터 출처 :https://www.kaggle.com/adityakadiwal/water-potability
데이터 설명 : 수질 음용성 여부 (Potablillity 컬럼 : 0 ,1 )
Data url :https://raw.githubusercontent.com/Datamanim/datarepo/main/waters/train.csv"""
# Q93. ph값은 상당히 많은 결측치를 포함한다. 결측치를 제외한 나머지 데이터들 중 사분위값 기준 하위 25%의 값들의 평균값은?

In [ ]:
"""데이터 출처 :https://www.kaggle.com/mirichoi0218/insurance/code(참고, 데이터 수정)
데이터 설명 : 의료비용 예측문제
Data url :https://raw.githubusercontent.com/Datamanim/datarepo/main/MedicalCost/train.csv"""
# Q94. 흡연자와 비흡연자 각각 charges의 상위 10% 그룹의 평균의 차이는?

In [ ]:
"""데이터 출처 :https://www.kaggle.com/harlfoxem/housesalesprediction (참고, 데이터 수정)
데이터 설명 : 킹카운티 주거지 가격 예측문제
Data url :https://raw.githubusercontent.com/Datamanim/datarepo/main/kingcountyprice/train.csv"""
# Q95. bedrooms 의 빈도가 가장 높은 값을 가지는 데이터들의 price의 상위 10%와 하위 10%값의 차이를 구하여라

In [ ]:
"""데이터 출처 :https://www.kaggle.com/mohansacharya/graduate-admissions(참고, 데이터 수정)
데이터 설명 : 대학원 입학 가능성 예측
Data url :https://raw.githubusercontent.com/Datamanim/datarepo/main/admission/train.csv"""
# Q96. Serial No. 컬럼을 제외하고 ‘Chance of Admit’을 종속변수, 나머지 변수를 독립변수라 할때,
#  랜덤포레스트를 통해 회귀 예측을 할 떄 변수중요도 값을 출력하라 (시드값에 따라 순서는 달라질수 있음)

In [ ]:
"""데이터 출처 :https://www.kaggle.com/uciml/red-wine-quality-cortez-et-al-2009(참고, 데이터 수정)
데이터 설명 : 레드 와인 퀄리티 예측문제
Data url :https://raw.githubusercontent.com/Datamanim/datarepo/main/redwine/train.csv"""
# Q97. quality 값이 3인 그룹과 8인 데이터그룹의 각 컬럼별 독립변수의 표준편차 값의 차이를 구할때
# 그값이 가장 큰 컬럼명을 구하여라

In [ ]:
"""데이터 출처 :https://www.kaggle.com/prathamtripathi/drug-classification(참고, 데이터 수정)
데이터 설명 : 투약하는 약을 분류
Data url :https://raw.githubusercontent.com/Datamanim/datarepo/main/drug/train.csv"""
# Q98. 남성들의 연령대별 (10살씩 구분 0~9세 10~19세 …) Na_to_K값의 평균값을 구해서 데이터 프레임으로 표현하여라

In [ ]:
"""데이터 출처 :https://www.kaggle.com/sid321axn/audit-data(참고, 데이터 수정)
데이터 설명 : 사기회사 분류
Data url :https://raw.githubusercontent.com/Datamanim/datarepo/main/audit/train.csv"""
# Q99. 데이터의 Risk 값에 따른 score_a와 score_b의 평균값을 구하여라

In [ ]:
"""데이터 출처 :https://www.kaggle.com/kyr7plus/emg-4(참고, 데이터 수정)
데이터 설명 : 센서데이터로 동작 유형 분류
Data url :https://raw.githubusercontent.com/Datamanim/datarepo/main/muscle/train.csv"""
# Q100. pose값에 따른 각 motion컬럼의 중간값의 가장 큰 차이를 보이는 motion컬럼은 어디이며 그값은?

In [ ]:
"""데이터 출처 :https://www.kaggle.com/mysarahmadbhat/hyundai-used-car-listing(참고, 데이터 수정)
데이터 설명 : 현대 차량가격 분류문제
Data url :hhttps://raw.githubusercontent.com/Datamanim/datarepo/main/hyundai/train.csv"""
# Q101. 정보(row수)가 가장 많은 상위 3차종의 price값의 각 평균값은?

In [ ]:
"""데이터 출처 :https://www.kaggle.com/pritsheta/diabetes-dataset(참고, 데이터 수정)
데이터 설명 : 당뇨여부 판단하기
Data url :https://raw.githubusercontent.com/Datamanim/datarepo/main/diabetes/train.csv"""
# Q102. Outcome 값에 따른 각 그룹의 각 컬럼의 평균 차이를 구하여라

In [ ]:
"""데이터 출처 :https://www.kaggle.com/pritsheta/netflix-stock-data-from-2002-to-2021(참고, 데이터 수정)
데이터 설명 : 넷플릭스 주식데이터
Data url :https://raw.githubusercontent.com/Datamanim/datarepo/main/nflx/NFLX.csv"""
# Q103. 매년 5월달의 open가격의 평균값을 데이터 프레임으로 표현하라

In [ ]:
"""데이터 출처 :https://www.kaggle.com/datasets/vivovinco/nba-player-stats(참고, 데이터 수정)
Data url :https://raw.githubusercontent.com/Datamanim/datarepo/main/nba/nba.csv"""
# Q104. Tm 컬럼은 각 팀의 이름을 의미한다. TOR팀의 평균나이를 소수 4째 자리까지 구하여라

In [ ]:
# Q105. Pos 컬럼은 포지션을 의미한다. 전체 선수 중 최소나이대의 선수들을 필터하고 그들 중 가장 많은 포지션은 무엇인지 확인하라

In [ ]:
# Q106. 선수들의 이름은 first_name+ 공백 + last_name으로 이루어져 있다.
# 가장 많은 first_name은 무엇이며 몇 회 발생하는지 확인하라

In [ ]:
# Q107. PTS컬럼은 경기당 평균득점수 이다. 각포지션별로 경기당 평균득점수의 평균을 구하여라

In [ ]:
# Q108. PTS컬럼은 경기당 평균득점수 이다. 각포지션별로 경기당 평균득점수의 평균을 구하여라

In [ ]:
# Q109. G컬럼은 참여한 경기의 숫자이다. 각 팀별로 가장 높은 경기참여 수를 가진 선수들의 경기 참여 숫자의 평균을 구하여라

In [ ]:
# Q110. Tm의 값이 MIA이며 Pos는 C또는 PF인 선수의 MP값의 평균은?

In [ ]:
# Q111. 전체 데이터중 G의 평균값의 1.5배 이상인 데이터들만 추출했을때 3P값의 평균은?

In [ ]:
# Q112. Age의 평균 이상인 그룹과 평균 미만인 그룹간의 G값의 평균의 차이는?

In [ ]:
# Q113. 평균나이가 가장 젊은 팀은 어디인가

In [ ]:
# Q114. Pos그룹별 평균 MP값을 구하여라